# Clustering Crypto

In [ ]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [102]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [103]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [104]:
# Alternatively, use the provided csv file:
file_path = Path('../Resources/crypto_data.csv')

# Create a DataFrame
crypto_df = pd.read_csv(file_path)
crypto_df.head(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


### Data Preprocessing

In [105]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_data =  crypto_df.drop(columns=['Unnamed: 0'])

In [106]:
# Keep only cryptocurrencies that are trading
crypto_data = crypto_data.loc[crypto_data['IsTrading'] == True]

In [107]:
# Keep only cryptocurrencies with a working algorithm
crypto_data = crypto_data.loc[crypto_data['Algorithm'] != 'N/A']

In [108]:
# Remove the "IsTrading" column
crypto_data = crypto_data.drop(columns='IsTrading')

In [109]:
# Remove rows with at least 1 null value
crypto_data = crypto_data.dropna()

In [110]:
# Remove rows with cryptocurrencies having no coins mined
crypto_data = crypto_data[crypto_data['TotalCoinsMined']!=0]

In [111]:
# Drop rows where there are 'N/A' text values
crypto_data = crypto_data.loc[crypto_data['TotalCoinSupply'] != 'N/A']
crypto_data = crypto_data.loc[crypto_data['ProofType'] != 'N/A']

In [112]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
all_crypto_data = crypto_data

In [113]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_data = crypto_data.drop(columns='CoinName')
crypto_data.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0
9,Scrypt,PoW,6.303924e+07,84000000
10,X11,PoW/PoS,9.031294e+06,22000000
11,CryptoNight-V7,PoW,1.720114e+07,0
12,Ethash,PoW,1.133597e+08,210000000
13,Equihash,PoW,7.383056e+06,21000000


In [126]:
# Create dummy variables for text features
crypto_dummy = pd.get_dummies(data=crypto_data, columns=['Algorithm', 'ProofType'])

In [128]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(crypto_dummy)
crypto_scaled

array([[-0.11674788, -0.15286468, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Reducing Dimensions Using PCA

In [129]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)

In [130]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_data.index
)
pca_df.head()

,PC 1,PC 2,PC 3
0,-0.339006,0.994550,-0.611084
2,-0.322323,0.994722,-0.611506
5,2.290929,1.702480,-0.533484
7,-0.149803,-1.325676,0.180493
8,-0.156427,-2.006796,0.397405


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [131]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Visualization", xlabel="Number of Clusters", ylabel="Inertia")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [120]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features


### Visualizing Results

#### 3D-Scatter with Clusters

In [121]:
# Create a 3D-Scatter with the PCA data and the clusters


#### Table of Tradable Cryptocurrencies

In [122]:
# Table with tradable cryptos


In [123]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [124]:
# Scale data to create the scatter plot


In [125]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
